In [2]:
import logging

logger = logging.getLogger(__name__)

In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from short_tracker.data import (
    query_all_sec_metadata, query_uk_si_disclosures, query_mkt_data, query_quotes,
    SHORT_URL_UK, DATE_COL, FUND_COL, ISIN_COL, SHORT_POS_COL,
    SHARE_ISSUER_COL, UK_DISCL_THRESHOLD, FCA_DATE_COL,
)
from short_tracker.processing import (
    check_cur_hist_discl_overlap, remove_dupl_shorts, ffill_discl_data, calc_fund_short_flow_bounds,
    extract_sec_tickers, process_mkt_data, subset_top_shorts,
)

NUM_SUBSET = 20
UK_MKT_TICKER = "VUKE"
ALPHAVANGATE_KEY = ""

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
discl_data, rept_date = query_uk_si_disclosures(SHORT_URL_UK) # exp_upd_time
rept_date

datetime.date(2022, 12, 30)

In [5]:
cur_discl = discl_data['current']

exp_max_discl_date = rept_date - timedelta(days=1)
max_discl_date = cur_discl[FCA_DATE_COL].max().date()

assert max_discl_date <= exp_max_discl_date
max_discl_date

datetime.date(2022, 12, 29)

In [6]:
# subset on current top N total overall shorts + top N individual shorts
top_sec_shorts, top_fund_shorts = subset_top_shorts(cur_discl, NUM_SUBSET)

isins = list({*top_fund_shorts[ISIN_COL].values, *top_sec_shorts[ISIN_COL].values})
print(len(isins))
isins[:5]

30


['GB0032089863',
 'GB00B132NW22',
 'GB0030927254',
 'GB00B4Y7R145',
 'GB0033195214']

In [7]:
sec_metadata, err_isins = query_all_sec_metadata(isins, {"idType": "ID_ISIN", "exchCode": "LN"})
len(err_isins)

0

In [8]:
isin_ticker_map = extract_sec_tickers(sec_metadata)
tickers = [UK_MKT_TICKER, *isin_ticker_map.values()]
query_tickers = [k.rstrip("/") + '.L' for k in tickers]

ticker_map = dict(zip(tickers, query_tickers))

tickers[:5]

['VUKE', 'NXT', 'ASHM', 'ASC', 'CURY']

In [60]:
cur_discl

,Position Holder,Name of Share Issuer,ISIN,Net Short Position (%),Position Date
0,BlackRock Investment Management (UK) Limited,abrdn plc,GB00BF8Q6K64,1.05,2022-12-16
1,Citadel Advisors Europe Limited,abrdn plc,GB00BF8Q6K64,0.90,2022-11-22
2,Citadel Advisors LLC,abrdn plc,GB00BF8Q6K64,1.00,2022-12-29
3,GLG Partners LP,abrdn plc,GB00BF8Q6K64,1.15,2022-12-16
4,Qube Research & Technologies Limited,abrdn plc,GB00BF8Q6K64,0.55,2022-12-16
...,...,...,...,...,...
295,GLG Partners LP,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.68,2022-11-11
296,Qube Research & Technologies Limited,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.81,2022-12-16
297,Marshall Wace LLP,WPP PLC,JE00B8KF9B49,0.79,2022-12-13
298,GLG Partners LP,XP Power Limited,SG9999003735,1.06,2022-11-10


In [ ]:
n_bdays_ago
rept_date

In [59]:
pd.Series(isin_ticker_map, name='ticker').rename_axis('isin').reset_index()

,isin,ticker
0,GB0032089863,NXT
1,GB00B132NW22,ASHM
2,GB0030927254,ASC
3,GB00B4Y7R145,CURY
4,GB0033195214,KGF
5,GB00B53P2009,JUP
6,GB00BYRJ5J14,PHP
7,GB00B019KW72,SBRY
8,GB00BJTNFH41,AO/
9,GB00BZ6STL67,MTRO


In [10]:
query_start = datetime.today().date() - timedelta(days=50)

mkt_data = {tkr: query_mkt_data(qry_tkr, query_start) for tkr, qry_tkr in ticker_map.items()}
quotes = {tkr: query_quotes(qry_tkr) for tkr, qry_tkr in ticker_map.items()}

In [11]:
SH_OUT_COL = 'shares_outstanding'

mkt_data_df = process_mkt_data(mkt_data)
quotes_df = pd.DataFrame(quotes)
sh_out = quotes_df.loc['sharesOutstanding'].rename(SH_OUT_COL)

missing_sh_out = list(sh_out[sh_out.isna()].index)

if missing_sh_out:
    logger.warning(f"No share outstanding data for tickers: {missing_sh_out}")

No share outstanding data for tickers: ['VUKE']


In [50]:
from short_tracker.utils import setup_logging, n_bdays_ago

In [53]:
sh_out_ = sh_out.rename_axis("ticker").rename('value').to_frame().reset_index()
sh_out_.loc[:, 'date'] = pd.to_datetime(n_bdays_ago(1))
sh_out_.loc[:, 'item'] = 'shout'

In [54]:
sh_out_

,ticker,value,date,item
0,VUKE,NaN,2022-12-30,shout
1,NXT,129263000,2022-12-30,shout
2,ASHM,712740992,2022-12-30,shout
3,ASC,100015000,2022-12-30,shout
4,CURY,1133490048,2022-12-30,shout
5,KGF,1939730048,2022-12-30,shout
6,JUP,546387008,2022-12-30,shout
7,PHP,1336489984,2022-12-30,shout
8,SBRY,2345060096,2022-12-30,shout
9,AO/,576899968,2022-12-30,shout


In [41]:
mkt_data_df.rename_axis(index=DATE_COL, columns='item').set_index("Ticker", append=True).stack().rename('value').reset_index()

,date,Ticker,item,value
0,2022-11-11,VUKE,Close,32.025002
1,2022-11-11,VUKE,Adj Close,32.025002
2,2022-11-11,VUKE,Volume,276196.000000
3,2022-11-14,VUKE,Close,32.305000
4,2022-11-14,VUKE,Adj Close,32.305000
...,...,...,...,...
3157,2022-12-29,VCP,Adj Close,494.000000
3158,2022-12-29,VCP,Volume,106818.000000
3159,2022-12-30,VCP,Close,481.000000
3160,2022-12-30,VCP,Adj Close,481.000000


datetime.date(2022, 12, 30)

In [15]:
mkt_data_df.assign(**{DATE_COL: rept_date})

,Close,Adj Close,Volume,Ticker,Position Date
2021-12-30,32.334999,32.334999,181089,VUKE,2022-12-30
2021-12-31,32.259998,32.259998,63784,VUKE,2022-12-30
2022-01-04,32.775002,32.775002,922919,VUKE,2022-12-30
2022-01-05,32.860001,32.860001,379070,VUKE,2022-12-30
2022-01-06,32.570000,32.570000,505078,VUKE,2022-12-30
...,...,...,...,...,...
2022-12-22,34.950001,34.950001,10006476,BOO,2022-12-30
2022-12-23,34.750000,34.750000,9713914,BOO,2022-12-30
2022-12-28,33.889999,33.889999,13647584,BOO,2022-12-30
2022-12-29,36.000000,36.000000,9983660,BOO,2022-12-30


In [13]:
cur_discl

,Position Holder,Name of Share Issuer,ISIN,Net Short Position (%),Position Date
0,BlackRock Investment Management (UK) Limited,abrdn plc,GB00BF8Q6K64,1.05,2022-12-16
1,Citadel Advisors Europe Limited,abrdn plc,GB00BF8Q6K64,0.90,2022-11-22
2,Citadel Advisors LLC,abrdn plc,GB00BF8Q6K64,1.00,2022-12-29
3,GLG Partners LP,abrdn plc,GB00BF8Q6K64,1.15,2022-12-16
4,Qube Research & Technologies Limited,abrdn plc,GB00BF8Q6K64,0.55,2022-12-16
...,...,...,...,...,...
295,GLG Partners LP,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.68,2022-11-11
296,Qube Research & Technologies Limited,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.81,2022-12-16
297,Marshall Wace LLP,WPP PLC,JE00B8KF9B49,0.79,2022-12-13
298,GLG Partners LP,XP Power Limited,SG9999003735,1.06,2022-11-10


In [ ]:
ALPHAVANTAGE_ENDPOINT = "https://www.alphavantage.co/query?"

import requests

from short_tracker.data import APIRateLimitException

# def query_mkt_data(ticker, api_key):
#     """Query Alphavantage for adjusted stock prices and """
#     def query_endpoint(fn):
#         av_args = {"function": fn, "symbol": ticker, "apikey": api_key}
#         resp = requests.get(ALPHAVANTAGE_ENDPOINT, params=av_args)
#         resp.raise_for_status()
#         data = resp.json()
        
#         if "Note" in data:
#             raise APIRateLimitException
#         elif "Error Message" in data:
#             raise ValueError(data["Error Message"])
#         return data
        
#     price = query_endpoint("TIME_SERIES_DAILY_ADJUSTED")
#     sec_info = query_endpoint("OVERVIEW")
#     return price, sec_info

# price, sec_info = query_mkt_data("RWI.L", ALPHAVANGATE_KEY)

In [18]:
# TODO: nice to have schemas defined like this but keeping things simple for now

from typing import List
from sqlalchemy import Column, Float, String, create_engine
# from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.ext.declarative import declarative_base
from pydantic import BaseModel, constr

engine = create_engine("sqlite:///:memory:")

Base = declarative_base()


class DisclData(Base):
    __tablename__ = 'discl_data'
    fund = Column(String(200), primary_key=True, nullable=False)
    isin = Column(String(20), primary_key=True, nullable=False)
    share_issuer = Column(String(200), primary_key=True, nullable=False)
    short_position = Float()
    
    class Config:
        orm_mode = True


co_orm = DisclData(fund='a', isin='b', share_issuer='c', short_position=1.1231)
print(co_orm)

In [21]:
md = Base.metadata
md.create_all(engine)

In [22]:
md

FacadeDict({'discl_data': Table('discl_data', MetaData(), Column('fund', String(length=200), table=<discl_data>, primary_key=True, nullable=False), Column('isin', String(length=20), table=<discl_data>, primary_key=True, nullable=False), Column('share_issuer', String(length=200), table=<discl_data>, primary_key=True, nullable=False), schema=None)})

In [23]:
cur_discl

NameError: name 'cur_discl' is not defined

In [20]:
pd.read_sql_query('select * from discl_data', con=engine)

,fund,isin,share_issuer


In [33]:
cur_discl.to_sql(name='discl_data', con=engine, if_exists='append')

OperationalError: (sqlite3.OperationalError) table discl_data has no column named index
[SQL: INSERT INTO discl_data ("index", "Position Holder", "Name of Share Issuer", "ISIN", "Net Short Position (%)", "Position Date") VALUES (?, ?, ?, ?, ?, ?)]
[parameters: ((0, 'BlackRock Investment Management (UK) Limited', 'abrdn plc', 'GB00BF8Q6K64', 1.05, '2022-12-16 00:00:00.000000'), (1, 'Citadel Advisors Europe Limited', 'abrdn plc', 'GB00BF8Q6K64', 0.9, '2022-11-22 00:00:00.000000'), (2, 'Citadel Advisors LLC', 'abrdn plc', 'GB00BF8Q6K64', 1.0, '2022-12-29 00:00:00.000000'), (3, 'GLG Partners LP', 'abrdn plc', 'GB00BF8Q6K64', 1.15, '2022-12-16 00:00:00.000000'), (4, 'Qube Research & Technologies Limited', 'abrdn plc', 'GB00BF8Q6K64', 0.55, '2022-12-16 00:00:00.000000'), (5, 'Bronte Capital Management Pty Ltd', 'AFC ENERGY PLC', 'GB00B18S7B29', 0.5, '2022-03-31 00:00:00.000000'), (6, 'AKO Capital LLP', 'Alphawave IP Group PLC', 'GB00BNDRMJ14', 0.66, '2022-10-11 00:00:00.000000'), (7, 'KUVARI

In [34]:
cur_discl

,Position Holder,Name of Share Issuer,ISIN,Net Short Position (%),Position Date
0,BlackRock Investment Management (UK) Limited,abrdn plc,GB00BF8Q6K64,1.05,2022-12-16
1,Citadel Advisors Europe Limited,abrdn plc,GB00BF8Q6K64,0.90,2022-11-22
2,Citadel Advisors LLC,abrdn plc,GB00BF8Q6K64,1.00,2022-12-29
3,GLG Partners LP,abrdn plc,GB00BF8Q6K64,1.15,2022-12-16
4,Qube Research & Technologies Limited,abrdn plc,GB00BF8Q6K64,0.55,2022-12-16
...,...,...,...,...,...
295,GLG Partners LP,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.68,2022-11-11
296,Qube Research & Technologies Limited,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.81,2022-12-16
297,Marshall Wace LLP,WPP PLC,JE00B8KF9B49,0.79,2022-12-13
298,GLG Partners LP,XP Power Limited,SG9999003735,1.06,2022-11-10


In [1]:
from short_tracker.data_main import main as data_main
data_main()

No share outstanding data for tickers: ['VUKE']
/home/nikul/Documents/Programming/short_tracker/src/short_tracker/data_main.py:130: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  mkt_data.loc[:, DATE_COL] = pd.to_datetime(mkt_data[DATE_COL])
/home/nikul/Documents/Programming/short_tracker/src/short_tracker/data_main.py:130: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  mkt_data.loc[:, DATE_COL] = pd.to_datetime(mkt_data[DATE_COL])
/home/nikul/Documents/Programming/short_tracker/src/short_tracker/data_main.py:130: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to se

In [1]:
from short_tracker.metrics import main as metrics_main
metrics_main()

In [1]:
%load_ext autoreload
%autoreload 2


from short_tracker.config import (
    CONN_STR,
    MAX_DATA_AGE,
    TOP_N_SHORTS,
    UK_MKT_TICKER,
    ADV_CALC_LOOKBACK,
)

from short_tracker.data import FUND_COL, ADJ_CLOSE_COL, SH_OUT_COL, VOLUME_COL, TICKER_COL, DATE_COL, ISIN_COL, ITEM_COL, VALUE_COL, RET_COL, REL_RET_COL, REL_PNL_COL, PNL_COL, BM_RET_COL, EXPO_COL, AMOUNT_COL, SHARE_ISSUER_COL, SHORT_POS_COL, DTC_COL
from short_tracker.processing import reindex_rename_mkt_data, calc_reindex_dates, calc_returns, prepare_mkt_data, prepare_discl_data, subset_top_shorts
from short_tracker.metrics import query_all_db_data, summarise_short_discl, augment_discl_metrics, style_metrics_df
from short_tracker.utils import n_bdays_ago
    
discl_data, mkt_data, isin_ticker_map = query_all_db_data()

sec_short_metrics, fund_short_metrics = summarise_short_discl(discl_data, mkt_data, isin_ticker_map)

In [15]:
from short_tracker.config import METRICS_LOOKBACK
from short_tracker.metrics import susbet_hist_disclosures, calc_display_metrics

latest_rpt_date = discl_data[DATE_COL].max()
lookback_date = n_bdays_ago(METRICS_LOOKBACK, latest_rpt_date)
reindex_dates = calc_reindex_dates(latest_rpt_date)
mkt_data_concat = prepare_mkt_data(mkt_data, reindex_dates, UK_MKT_TICKER)

discl_data_ = prepare_discl_data(discl_data, isin_ticker_map)

dt_cond = (discl_data_[DATE_COL] <= latest_rpt_date) & (
    discl_data_[DATE_COL] >= lookback_date
)
lookback_discl = discl_data_[dt_cond]

top_sec_shorts_lookback, top_fund_shorts_lookback = susbet_hist_disclosures(
    lookback_discl, TOP_N_SHORTS, latest_rpt_date
)
top_sec_shorts_lookback_ = top_sec_shorts_lookback.merge(
    mkt_data_concat, on=[DATE_COL, TICKER_COL], how="left"
)
top_fund_shorts_lookback_ = top_fund_shorts_lookback.merge(
    mkt_data_concat, on=[DATE_COL, TICKER_COL], how="left"
)

fund_short_metrics = calc_display_metrics(top_fund_shorts_lookback_)
sec_short_metrics = calc_display_metrics(top_sec_shorts_lookback_)

In [16]:
sec_short_metrics

,Name of Share Issuer,Net Short Position (%),Return,PnL,Days to Cover,Exposure,Position Change (%),Lookback Date
Ticker,,,,,,,,
ABDN,ABRDN PLC,0.000005,0.007382,-156.054354,0.000857,-20458.466805,-7.000000e-07,2022-12-30
ASC,ASOS PLC,0.000007,0.123445,-475.086252,0.001016,-4258.898739,1.000000e-08,2022-12-30
BOO,Boohoo Group,0.000005,0.082222,-193.292380,0.000608,-2563.823495,4.000000e-08,2022-12-30
CURY,CURRYS PLC,0.000004,0.109890,-283.916586,0.001156,-2958.091611,-8.000000e-08,2022-12-30
HMSO,HAMMERSON PLC,0.000004,0.058403,-298.525742,0.002513,-5324.062235,0.000000e+00,2022-12-30
ITM,ITM Power,0.000006,0.040687,-140.306719,0.001600,-3499.625911,0.000000e+00,2022-12-30
KGF,KINGFISHER PLC,0.000006,0.066919,-1791.978716,0.001714,-28645.817845,-6.000000e-08,2022-12-30
MTRO,METRO BANK PLC,0.000004,-0.054487,47.751638,0.000746,-817.395335,0.000000e+00,2022-12-30
TPK,TRAVIS PERKINS PLC,0.000004,0.097201,-794.073410,0.001168,-8766.114709,1.100000e-07,2022-12-30


In [14]:
a[
    (a['Position Holder']=='Ennismore Fund Management Limited')
    &(a['ISIN']=='GB00B021F836')
    ]

,ISIN,Position Holder,Name of Share Issuer,date,Net Short Position (%),Ticker,sharesOutstanding,Adj Close,Return,Volume,Benchmark Return
6,GB00B021F836,Ennismore Fund Management Limited,Naked Wine,2023-01-06,0.00024,WINE,74004096.0,1.346,0.003715,773399.363636,0.008704
16,GB00B021F836,Ennismore Fund Management Limited,Naked Wine,2023-01-05,0.00024,WINE,74004096.0,1.360,-0.010294,774957.727273,0.005735
26,GB00B021F836,Ennismore Fund Management Limited,NAKED WINES PLC,2023-01-04,0.00024,WINE,74004096.0,1.344,0.011905,773940.500000,0.003939
36,GB00B021F836,Ennismore Fund Management Limited,NAKED WINES PLC,2023-01-03,0.00024,WINE,74004096.0,1.268,0.059937,769085.090909,0.011184
46,GB00B021F836,Ennismore Fund Management Limited,NAKED WINES PLC,2022-12-30,0.00024,WINE,74004096.0,1.290,-0.017054,756845.909091,-0.005182


In [6]:
discl_data[
    (discl_data['Position Holder']=='BlackRock Investment Management (UK) Limited')
    &(discl_data['ISIN']=='GB00BF8Q6K64')
    ]

,ISIN,Position Holder,Name of Share Issuer,date,Net Short Position (%)
0,GB00BF8Q6K64,BlackRock Investment Management (UK) Limited,ABRDN PLC,2023-01-06,0.0105
303,GB00BF8Q6K64,BlackRock Investment Management (UK) Limited,ABRDN PLC,2023-01-05,0.0105
606,GB00BF8Q6K64,BlackRock Investment Management (UK) Limited,abrdn plc,2023-01-04,0.0105
910,GB00BF8Q6K64,BlackRock Investment Management (UK) Limited,abrdn plc,2023-01-03,0.0105
1210,GB00BF8Q6K64,BlackRock Investment Management (UK) Limited,abrdn plc,2022-12-30,0.0105


In [2]:
sec_short_metrics

Net Short Position (%)    Return           PnL  \
Ticker Name of Share Issuer                                                   
ABDN   ABRDN PLC                             0.0535 -0.003910  8.058745e+05   
       abrdn plc                             0.0535  0.011337 -2.366418e+06   
ASC    ASOS PLC                              0.0738  0.123445 -4.750863e+06   
BOO    BOOHOO GROUP PLC                      0.0514  0.057778 -1.355848e+06   
       Boohoo Group                          0.0510  0.023109 -5.770754e+05   
CURY   CURRYS PLC                            0.0424  0.018487 -5.286590e+05   
       Currys plc                            0.0416  0.089744 -2.310507e+06   
HMSO   HAMMERSON PLC                         0.0430  0.058403 -2.985257e+06   
ITM    ITM POWER PLC                         0.0593  0.044289 -1.527296e+06   
       ITM Power                             0.0593 -0.003450  1.242284e+05   
KGF    KINGFISHER PLC                        0.0581  0.066919 -1.791979e+07   
MTRO   METRO BANK PLC                        0.0407 -0.054487  4.775164e+05   
TPK    TRAVIS PERKINS PLC                    0.0416  0.097201 -7.940734e+06   
WINE   NAKED WINES PLC                       0.0579  0.054264 -2.999386e+05   
       Naked Wine                            0.0579 -0.006618  3.856327e+04   

                             Days to Cover      Exposure  Position Change (%)  \
Ticker Name of Share Issuer                                                     
ABDN   ABRDN PLC                  8.566584 -2.045847e+08               0.0000   
       abrdn plc                  7.971030 -2.018447e+08              -0.0070   
ASC    ASOS PLC                  10.163942 -4.258899e+07               0.0001   
BOO    BOOHOO GROUP PLC           6.135604 -2.406629e+07               0.0000   
       Boohoo Group               6.079497 -2.563823e+07               0.0004   
CURY   CURRYS PLC                11.556912 -2.958092e+07               0.0000   
       Currys plc                11.239538 -2.680659e+07               0.0000   
HMSO   HAMMERSON PLC             25.131406 -5.324062e+07               0.0000   
ITM    ITM POWER PLC             13.474714 -3.541279e+07               0.0000   
       ITM Power                 15.997565 -3.499626e+07               0.0000   
KGF    KINGFISHER PLC            17.141188 -2.864582e+08              -0.0006   
MTRO   METRO BANK PLC             7.461567 -8.173953e+06               0.0000   
TPK    TRAVIS PERKINS PLC        11.675614 -8.766115e+07               0.0011   
WINE   NAKED WINES PLC            5.536391 -5.758821e+06               0.0000   
       Naked Wine                 5.540265 -5.767391e+06               0.0000   

                            Lookback Date  
Ticker Name of Share Issuer                
ABDN   ABRDN PLC               2023-01-05  
       abrdn plc               2022-12-30  
ASC    ASOS PLC                2022-12-30  
BOO    BOOHOO GROUP PLC        2022-12-30  
       Boohoo Group            2023-01-05  
CURY   CURRYS PLC              2023-01-05  
       Currys plc              2022-12-30  
HMSO   HAMMERSON PLC           2022-12-30  
ITM    ITM POWER PLC           2022-12-30  
       ITM Power               2023-01-05  
KGF    KINGFISHER PLC          2022-12-30  
MTRO   METRO BANK PLC          2022-12-30  
TPK    TRAVIS PERKINS PLC      2022-12-30  
WINE   NAKED WINES PLC         2022-12-30  
       Naked Wine              2023-01-05

In [5]:
report_date = discl_data[DATE_COL].max()
df = fund_short_metrics

st = (
    style_metrics_df(df, report_date)
    # .set_table_styles([{"selector": "tr:hover", "props": "background-color: yellow"}])
)

with open('/home/nikul/Documents/tmp/index.html', 'w') as f:
    f.write(st.to_html())

st

Ticker,Name of Share Issuer,Position Holder,Net Short Position (%),Return,PnL,Days to Cover,Exposure,Exposure Change,Lookback Date
MTRO,METRO BANK PLC,ODEY ASSET MANAGEMENT LLP,4.1%,-3.0%,£267k,7.6,-£8.8M,£0k,22-12-30
FEVR,FEVERTREE DRINKS PLC,BlackRock Investment Management (UK) Limited,4.0%,-2.6%,"£1,249k",14.4,-£49.0M,£0k,22-12-30
ASC,ASOS PLC,Marshall Wace LLP,3.0%,-2.3%,£362k,3.9,-£15.8M,£0k,22-12-30
SBRY,SAINSBURY (J) PLC,BlackRock Investment Management (UK) Limited,2.6%,-0.6%,£840k,9.8,-£131.5M,£0k,22-12-30
VCP,VICTORIA PLC,"Coltrane Asset Management, L.P.",2.4%,-2.6%,£360k,7.9,-£13.7M,£0k,22-12-30
WINE,NAKED WINES PLC,Ennismore Fund Management Limited,2.4%,-1.7%,£39k,2.3,-£2.3M,£0k,22-12-30
EMIS,EMIS GROUP PLC,Beryl Capital Mangement LLC,2.3%,0.1%,-£29k,4.4,-£27.2M,£0k,22-12-30
ITM,ITM POWER PLC,Helikon Investments Limited,2.1%,-2.9%,£361k,4.4,-£12.4M,£0k,22-12-30
DTY,DIGNITY PLC,Ennismore Fund Management Limited,2.0%,-2.1%,£91k,5.7,-£4.3M,£0k,22-12-30
RWI,Renewi Plc,GLG Partners LP,2.0%,1.2%,-£110k,7.0,-£9.3M,£0k,22-12-30


In [46]:
import pandas as pd
from short_tracker.data_main import query_mkt_data_, query_uk_si_disclosures_, query_ticker_map

cur_discl, report_date = query_uk_si_disclosures_()

top_shorts = subset_top_shorts(cur_discl, TOP_N_SHORTS)
isins = pd.concat([k[ISIN_COL] for k in top_shorts]).unique()


isin_ticker_map = query_ticker_map(isins)
tickers = [UK_MKT_TICKER, *isin_ticker_map.values()]

mkt_data = query_mkt_data_(tickers, report_date)

No share outstanding data for tickers: ['VUKE']
/home/nikul/Documents/Programming/short_tracker/src/short_tracker/data_main.py:130: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  mkt_data.loc[:, DATE_COL] = pd.to_datetime(mkt_data[DATE_COL])
/home/nikul/Documents/Programming/short_tracker/src/short_tracker/data_main.py:131: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  mkt_data.loc[:, VALUE_COL] = pd.to_numeric(mkt_data[VALUE_COL])


In [54]:
from short_tracker.data_main import MKT_DATA_TABLE, SHORT_DISCL_TABLE, concat_old_new_data, create_engine, CONN_STR

engine = create_engine(CONN_STR)

start_date = n_bdays_ago(MAX_DATA_AGE, report_date)

isin_ticker_map_df = (
    pd.Series(isin_ticker_map, name=TICKER_COL).rename_axis(ISIN_COL).reset_index()
)

existing_shout_query = f"""
select * from {MKT_DATA_TABLE}
where item = '{SH_OUT_COL}' and
{DATE_COL} >= '{start_date}'
"""

existing_discl_query = f"""
select * from {SHORT_DISCL_TABLE}
where {DATE_COL} >= '{start_date}'
"""

existing_shout = pd.read_sql_query(
    existing_shout_query, con=engine, parse_dates=[DATE_COL]
)
existing_discl = pd.read_sql_query(
    existing_discl_query, con=engine, parse_dates=[DATE_COL]
)

upl_mkt_data = concat_old_new_data(
    mkt_data, existing_shout, start_date, [DATE_COL, ITEM_COL, TICKER_COL]
)
upl_discl_data = concat_old_new_data(
    discl_data,
    existing_discl,
    start_date,
    [DATE_COL, FUND_COL, ISIN_COL],
)

In [101]:
style_metrics_df(fund_short_metrics.head(5)[DISPL_COLS])

In [449]:
from short_tracker.metrics import main as metrics_main
metrics_main()

TypeError: Object of type DataFrame is not JSON serializable